# Calculating OLS

This exercise will walk you through calculating a multivariate ordinary linear regression.

## Housekeeping

In [1]:
using Distributions

In order to make the results replicable set the seed.

In [2]:
srand(0);

In this step we define the following parameters

In [3]:
n = 100; # Number of observations
beta1 = 2; # Weights on X1 and X2
beta2 = 1;
beta3 = Dict([("A",0),("B",-1.5),("C",3)])
IncludeIntercept = 1; # Should the intercept be included? (0 no, 1 yes)
print()

## Generating the data

Let us generate two features using *n* random numbers from a standard normal distribution: $X_{1}$ and $X_{2}$.

In [4]:
X1 = randn(n);
X2 = randn(n);
X3 = sample(["A","B","C"], 100);

Let us generate a random normal response (outcome variable) using a linear combination of $X_{1}$, $X_{2}$, and noise in the form of random draws from a standard normal distribution.

In [5]:
y = beta1 * X1 + beta2 * X2 + map(key -> beta3[key], X3) + randn(n);

To verify output with other software one can export the data as a text file.

In [6]:
writedlm("/Users/jbsc/Desktop/JuliaData.csv", cat(2,y,X1,X2,X3), ",")

Expand categorical features

In [7]:
"""
This function expands a categorical variable to the associated indicator matrix.
"""
function expand_categorical(variable)
    uniques = sort(unique(variable))
    output = zeros(length(variable), (length(uniques) - 1))
    for (index, value) in enumerate(uniques[2:length(uniques)])
        output[find(elem -> elem .== value, variable),index] = 1
    end
    return output
end

expand_categorical

In [8]:
X3 = expand_categorical(X3);

## Fitting the Model

In order to estimate $\beta$ we can use the normal equation solution.

Let us create the design matrix with the intercept.

In [9]:
X = cat(2, X1, X2, X3);
if IncludeIntercept == 1
    X = cat(2, ones(n,1), X);
end
print()

In [10]:
Xt = X';
inv_Xt_X = inv(Xt * X);

In [11]:
β = inv_Xt_X * Xt * y

5-element Array{Float64,1}:
 -0.0401132
  2.0813   
  0.97727  
 -1.64356  
  3.01109  

Calculate the predictions $\hat{y}$

In [12]:
yhat = X * β;

Calculate the residuals

In [13]:
res = y - yhat;

The sum of squares residuals are decomposed by source: *Model* and *Residual*

The degrees of freedoms are given by: ESS (k), RSS (n - k - intercept), TSS (n - intercept)

In [14]:
mdf = size(X,2) - IncludeIntercept
rdf = n - size(X,2)
tdf = n - IncludeIntercept
print("mdf: ", mdf, "\n")
print("rdf: ", rdf, "\n")
print("tdf: ", tdf, "\n")

mdf: 4
rdf: 95
tdf: 99


The Total Sum of Squared Residuals is defined as: $TSS = \Sigma\ (y_{i} - \bar{y}) ^ 2$

In [15]:
TSS = sum((y - mean(y)) .^ 2)

1025.1388643440375

In [16]:
MTSS = TSS / tdf

10.354938023677146

The Explained Sum of Squared Residuals (ESS) is defined as: $ESS = \Sigma (\hat{y}_{i} - \bar{y})^2$

In [17]:
ESS = sum((yhat - mean(y)) .^ 2)

924.2366986072334

In [18]:
MESS = ESS / mdf

231.05917465180835

The Residual Sum of Squared Residuals by the model (RSS) is defined as: $RSS = \Sigma (\hat{y}_{i} - y_{i}) ^ 2 = e^{\prime}e$

In [19]:
RSS = res' * res

100.90216573680424

In [20]:
MRSS = RSS / rdf

1.062128060387413

Calculate RMSE

In [21]:
RMSE = sqrt(MRSS)

1.0305959733995729

The coefficient of determination (R-squared) is defined as: $R^{2} = 1 - \frac{RSS}{TSS}$

In [22]:
Rsq = 1 - RSS / TSS

0.9015721974394472

$\beta$ is the expected values of the random variables the model estimated.

Standard errors are the measure of uncertainty associated with the estimates of the expected value of parameters.

$Var\left(\hat{\beta}\right) = \hat{\sigma} ^ 2 \left(X^{\prime}X\right)^{-1}$

$\hat{\sigma}$ is estimated from the Mean Residual Sum of Squared Residuals (MRSS).

In [23]:
se = sqrt.(MRSS * diag(inv_Xt_X))

5-element Array{Float64,1}:
 0.180163 
 0.100119 
 0.0986498
 0.260829 
 0.248988 

The t-statistics are calculated.

In [24]:
t_statistic = β ./ se

5-element Array{Float64,1}:
 -0.222649
 20.7884  
  9.90645 
 -6.3013  
 12.0933  

P-values are calculated using a t-distribution with $n - k - Intercept$ degrees of freedom

In [25]:
t_dist = TDist(rdf)

Distributions.TDist{Float64}(ν=95.0)

For a two-tailed test, the p-value is twice the area from the t-statistic to the tail.

In [26]:
p_value = 2 * ccdf(t_dist, abs.(t_statistic))

5-element Array{Float64,1}:
 0.824287   
 4.01682e-37
 2.61754e-16
 9.18322e-9 
 6.20306e-21

To calculate the F-test for the model use the F ratio: $F = \frac{MESS}{MRSS}$ and FDist( k , n - k - Intercept )

In [27]:
F = MESS / MRSS

217.5436119892446

In [30]:
F_dist = FDist(mdf, rdf)

Distributions.FDist{Float64}(ν1=4.0, ν2=95.0)

In [31]:
F_value = ccdf(F_dist, F)

6.528505057098376e-47

In [32]:
"""
This function calculates confidence intervals given a Distribution, mean (μ), standard deviation (σ), and confidence level.
This function can also be used for estimates using the sample statistics.
"""
function conf_intervals(Distribution, μ, σ, conf_level = 0.95)
    α = 1 - conf_level
    tstar = quantile(Distribution, 1 - α / 2)
    l, u = μ + [-1, 1] * tstar * σ
    return(l,u)
end

conf_intervals

In [33]:
CI = Array{Float64}(length(β), 2)
for idx in 1:size(CI,1)
    CI[idx,1], CI[idx,2] = conf_intervals(t_dist, β[idx], se[idx])
end

In [34]:
CI

5×2 Array{Float64,2}:
 -0.397782   0.317556
  1.88254    2.28006 
  0.781425   1.17311 
 -2.16138   -1.12575 
  2.51678    3.50539 

## Output

In [35]:
using DataFrames

In [36]:
Results = DataFrame(cat(2,["Intercept", "X1", "X2", "B", "C"],round.(cat(2,β,se,t_statistic,p_value,CI), 4)))
names!(Results, [:Feature, :Coeff, :Std_Error, :t_Statistic, :p_value, :LowerCI, :UpperCI])
Results

,Feature,Coeff,Std_Error,t_Statistic,p_value,LowerCI,UpperCI
1,Intercept,-0.0401,0.1802,-0.2226,0.8243,-0.3978,0.3176
2,X1,2.0813,0.1001,20.7884,0.0,1.8825,2.2801
3,X2,0.9773,0.0986,9.9065,0.0,0.7814,1.1731
4,B,-1.6436,0.2608,-6.3013,0.0,-2.1614,-1.1258
5,C,3.0111,0.249,12.0933,0.0,2.5168,3.5054


In [39]:
Anova = DataFrame(cat(2, ["Model", "Residual", "Total"], round.(cat(2, [ESS, RSS, TSS], [mdf, rdf, tdf], [MESS, MRSS, MTSS]), 4)))
names!(Anova, [:Source, :SS, :df, :MS])
Anova

,Source,SS,df,MS
1,Model,924.2367,4.0,231.0592
2,Residual,100.9022,95.0,1.0621
3,Total,1025.1389,99.0,10.3549


In [40]:
Model = DataFrame(cat(2, ["Sample Size", "R-Squared", "F-Statistic", "F-value", "RMSE"], round.([n, Rsq, F, F_value, RMSE],4)))
names!(Model, [:Statistic, :Value])
Model

,Statistic,Value
1,Sample Size,100.0
2,R-Squared,0.9016
3,F-Statistic,217.5436
4,F-value,0.0
5,RMSE,1.0306


## Diagnostics

### Multicollinearity through Variance Inflation Factor (VIF)

In [42]:
"""
VIF computes the sqrt VIF and mean sqrt VIF for all features.
"""
function VIF(X, IncludeIntercept)
    output = Vector{Float64}((size(X,2) - IncludeIntercept) + 1)
    for idx in 2:size(X,2)
        y = X[:,idx]
        x = X[:,setdiff(1:size(X,2),idx)]
        β = inv(x' * x) * x' * y
        yhat = x * β
        res = y - yhat
        RSS = res' * res
        TSS = sum((y - mean(y)) .^ 2)
        Rsq = 1 - RSS / TSS
        vif = 1 / ( 1 - Rsq)
        output[idx  - 1] = sqrt(vif)
    end
    output[length(output)] = mean(output[1:(length(output) - 1)])
    return output
end

VIF

In [43]:
VIF(X, IncludeIntercept)
vif = DataFrame(cat(2,["X1", "X2", "B", "C", "Mean sqrt VIF"], round.(VIF(X, IncludeIntercept), 4)))
names!(vif, [:Statistic, :Value])

,Statistic,Value
1,X1,1.021
2,X2,1.0122
3,B,1.1598
4,C,1.1597
5,Mean sqrt VIF,1.0882
